## DSCI 524 - Collaborative Software Development

### Lecture 5: Introductions to Continuous Integration (CI) & GitHub Actions

#### 2020-03-09

## Lecture 5 learning objectives:
By the end of this lecture, students should be able to:
- Define continuous integration testing
- Explain why continuous integration testing is superior to manually running tests
- Use GitHub Actions to set-up automated running of tests by `pytest` upon push to the master branch
- Use GitHub Actions to set-up automated running of tests by `testthat` upon push to the master branch

## Continuous Integration (CI)

Defined as the practice of frequently integrating code changes from contributors to a shared repository. Often the submission of code to the shared repository is combined with automated testing (and other things, such as style checking) to increase code dependability and quality.

## GitHub actions

A tool and service for automating software development tasks, located in the same place where you already store your code.

### Definitions:
**Actions:** Individual tasks you want to perform.

**Workflow:** A collection of actions (specified together in one file).

**Event:** Something that triggers the running of a workflow.

**Runner**: A machine that can run the Github Action(s).

**Step**: A set of commands or actions which a job executes.

### Curated list of awesome actions to use on GitHub

- <https://github.com/sdras/awesome-actions>